In [1]:
import os # 引入文件管理模块，读写文件，资源管理器
import time # 时间模块
import json # json文件读写模块
import serial # 串口通讯模块，pip install pyserial
import subprocess
# pip install paho-mqtt
import paho.mqtt.client as mqtt # MQTT(Message Queuing Telemetry Transport)是一种轻量级的即时通信协议，物联网，连接一些信号差，数据量少的设备
import sys # 操作系统模块，《操作系统基础》

In [2]:
SCREEN_DISPLAY = True
SAVE_TO_FILE = True
PUBLISH_TO_THINGSBOARD = False
DELIMITER = ','
SLEEP_TIME_SECONDS = 60*30 # 暂停时间，等待时间
SERIAL_PORT = '/dev/ttyS0' # 调用串口的接口，/dev linux操作系统中设备管理的目录，/ttyS0接口号
SERIAL_BAUD = 9600 # 通信带宽，波特率

In [23]:
file_name = 'yarwun_column.csv' # 读数据文件，将数据写入保存到这个文件

field_name = ['scale',
            'ec5',
            '555 K=0.05 R=21k #3',
            '555 K=0.05 R=21k #1',
            '555 K=0.05 R=21k #2', 
            '555 K=0.05 R=20k',
            '555 K=0.05 R=21k #4',
            '555 K=0.05 R=21k #5',
            'temp',
            'humidity'] # 字段名或者数据文件头

data_collected = dict((el,0.0) for el in field_name) # 将列表转为字典，并将value初始化为0，字段名为key

In [4]:
data_collected

{'scale': 0.0,
 'ec5': 0.0,
 '555 K=0.05 R=21k #3': 0.0,
 '555 K=0.05 R=21k #1': 0.0,
 '555 K=0.05 R=21k #2': 0.0,
 '555 K=0.05 R=20k': 0.0,
 '555 K=0.05 R=21k #4': 0.0,
 '555 K=0.05 R=21k #5': 0.0,
 'temp': 0.0,
 'humidity': 0.0}

In [3]:
if (PUBLISH_TO_THINGSBOARD): # 初值为False所以不会运行内部代码，
    with open('/home/pi/pyduino/credential/testbench_basin.json') as f: # 读文件testbench_basin.json到变量f
        credential = json.load(f) # 使用json模块格式化文件内容

    try: # 错误处理方式，如果try内的代码运行错误，特别是读写文件、连接数据库、读写接口等，出现文件错误，连接中断时，运行except中的代码
        client = mqtt.Client() # 连接mqtt数据库
        client.username_pw_set(credential['access_token']) # 配置连接属性，证书属性值
        client.connect(credential['thingsboard_host'], 1883, 60) # 主机地址属性
        client.loop_start() # 开始连接并保存连接状态
    except Exception:
        print("Failed to connect to thingsboard") # 如果连接数据库失败，在屏幕输出Failed to connect to thingsboard

In [12]:
time_now = time.time()  # 使用time模块的时间函数
time.localtime(time_now)  # 获取当前系统时间

time.struct_time(tm_year=2022, tm_mon=9, tm_mday=3, tm_hour=8, tm_min=35, tm_sec=59, tm_wday=5, tm_yday=246, tm_isdst=0)

In [13]:
time_now # 从**年到现在的时间，用秒做单位

1662194159.6243582

In [15]:
int(round(time_now * 1000))

1662194159624

In [16]:
time_now = time.time()  # 使用time模块
time_now_local = time.localtime(time_now)  # 获取当前设备的系统时间
seconds_since_epoch = int(round(time_now * 1000))
time_now_local_str = time.strftime("%Y-%m-%d %H:%M:%S", time_now_local)

In [17]:
time_now_local_str

'2022-09-03 08:40:09'

In [21]:
if not os.path.isfile('testbench_basin.json'):
    print('json')

json


In [28]:
file_name

'yarwun_column.csv'

In [34]:
if not os.path.isfile(file_name):  # os.path.isfile()判断文件夹下是否有该文件，如果没有则运行
    f = open(file_name, 'a', 0)  # 如果没有该文件，则新建一个
    # 将字符内容保存到文件中
    f.write(
        "Timestamp,Temp (C),Humidity (%),Analog1,Analog2,Analog3,Analog4,Analog5,Analog6,Analog7,Analog8,Analog9,Analog10\r\n")
    f.close()

In [ ]:
try:    
    # while True:
    ard = serial.Serial(port=SERIAL_PORT, baudrate=SERIAL_BAUD, timeout=60) # 使用前面串口的属性，连接设备
    time.sleep(5) # 等待5秒
    
    time_now = time.time() # 使用time模块
    time_now_local = time.localtime(time_now) # 获取当前设备的系统时间
    seconds_since_epoch = int(round(time_now * 1000))
    time_now_local_str = time.strftime("%Y-%m-%d %H:%M:%S", time_now_local) # 格式化时间，将字符串数据持久化
    # 数据持久化指的就是将数据保存为文件（csv、json）或写入数据库长期保存下来的意思

    # Open file for appending
    if SAVE_TO_FILE: # 如果SAVE_TO_FILE=True则运行内部代码
        if not os.path.isfile(file_name): # os.path.isfile()判断文件夹下是否有该文件，如果没有则运行
            fid = open(file_name, 'a', 0) # 如果没有该文件，则新建一个
            # 将字符内容保存到文件中
            fid.write("Timestamp,Temp (C),Humidity (%),Analog1,Analog2,Analog3,Analog4,Analog5,Analog6,Analog7,Analog8,Analog9,Analog10\r\n") # Allocate column names
        fid = open(file_name, 'a', 0) # 如果当前文件夹已经有'yarwun_column.csv'文件了，'a'代表文件的属性为可保存的文档
        # 'r'标识文件为只读文件，'w'表示文件为可以写入的状态
        # fid.write(time_now_local_str + '\r\n')
        fid.write(time_now_local_str)   # 将格式化后的时间字符保存到yarwun_column.csv文件里

    if SCREEN_DISPLAY: # 如果SCREEN_DISPLAY为True
        print(time_now_local_str) # 在屏幕上面显示当前时间，很多时候系统在工作时如果没有输出，我们是不知道系统当前的工作状态的，系统是否正在正常工作
        # 如果系统时间长时间没有发生变化，则表示系统死机了或出了某些问题

    #---------------------------------Temp & Hum DHT22-----------------------------
    # DHT22 temp and humidity onboard
    # 从串口根据标识码获取数据并保存到fid中
    try:
        ard.write("dht22,54,power,2,points,3,dummies,1,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read = msg.split(',')[0:-1]
        dht22_temp = float(current_read[-2])
        dht22_hum = float(current_read[-1])
        data_collected['temp'] = dht22_temp
        data_collected['humidity'] = dht22_hum
        if SCREEN_DISPLAY: # 将获取的传感器信息显示到屏幕里
            print("Ambient Temp (C)" + ": " + str(dht22_temp))
            print("Ambient Humdity (%): " + str(dht22_hum))

        if SAVE_TO_FILE: # 将传感器传递的数据保存到fid变量里
            fid.write(DELIMITER + str(dht22_temp))
            fid.write(DELIMITER + str(dht22_hum))
            # fid.write(time_now_local_str + DELIMITER + 'Temp' + DELIMITER + "C" + DELIMITER+DELIMITER + str(dht22_temp) + '\r\n')
            # fid.write(time_now_local_str + DELIMITER + 'Humidity' + DELIMITER + "%" + DELIMITER+DELIMITER + str(dht22_hum) + '\r\n')
    except Exception: # 如果获取数据失败，则在屏幕上输出下面内容
        if SCREEN_DISPLAY:
            print("DHT22 reading failed, error: {0}".format(sys.exc_info()[0]))


    #---------------------------Moisture Sensor 1--------------
    # MS555 K = 0.05, R = 20k
    try:
        field_name_id = 5
        ard.write("analog,14,power,44,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline() # readline()一行行的读，碰到换行符则表示另一行
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=20k'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 K=0.05 R=20k reading failed, error: {0}".format(sys.exc_info()[0]))
        

    #----------------------------Moisture sensor 2---------------------------------
    # MS555 k=0.05 R=21k #3
    try:
        field_name_id = 2
        ard.write("analog,13,power,43,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read = msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #3'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #3 reading failed, error: {0}".format(sys.exc_info()[0]))


    #----------------------------Moisture sensor 3---------------------------------
    # MS555 k=0.05 R=21k #2
    try:
        field_name_id = 4
        ard.write("analog,11,power,41,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #2'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #2 reading failed, error: {0}".format(sys.exc_info()[0]))


    #---------------------------Moisture Sensor 4--------------------------------
    # MS555 k=0.05 R=21k #1
    try:
        field_name_id = 3
        ard.write("analog,10,power,40,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #1'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #1 reading failed, error: {0}".format(sys.exc_info()[0]))

    #--------------------------Moisture Sensor 5-------------------------------
    # EC5
    try:
        field_name_id = 1
        ard.write("analog,12,power,42,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['ec5'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("EC5 reading failed, error: {0}".format(sys.exc_info()[0]))


    #---------------------------Moisture Sensor 6--------------------------------
    # MS555 k=0.05 R=21k #4
    try:
        field_name_id = 6
        ard.write("analog,9,power,39,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #4'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #4 reading failed, error: {0}".format(sys.exc_info()[0]))

    #---------------------------Moisture Sensor 7--------------------------------
    # MS555 k=0.05 R=21k #5
    try:
        field_name_id = 7
        ard.write("analog,8,power,38,points,3,interval_mm,200,debug,0")
        ard.flushInput()
        msg = ard.readline()
        current_read=msg.split(',')[0:-1]
        data_collected['555 K=0.05 R=21k #5'] = float(current_read[2])
        if SCREEN_DISPLAY: print(field_name[field_name_id] + ": " + msg.rstrip())
        if SAVE_TO_FILE:
            fid.write(DELIMITER + current_read[2])
            # fid.write(time_now_local_str + DELIMITER + field_name[field_name_id] + DELIMITER + msg.rstrip() + '\r\n')
    except Exception:
        if SCREEN_DISPLAY:
            print("MS555 k=0.05 R=21k #4 reading failed, error: {0}".format(sys.exc_info()[0]))


    #----------------------------Upload data -----------------------------------    
    if (ard.isOpen()):
        ard.close()

    if (PUBLISH_TO_THINGSBOARD):
        json_data = {"ts":seconds_since_epoch, "values": data_collected}
        # json_data = data_collected
        try:
            # Result is in tuple (rc, mid) of MQTTMessageInfo class
            publish_result = client.publish('v1/devices/me/telemetry', payload=json.dumps(json_data), qos=1)
            # publish_result.wait_for_publish(timeout=1)
            # print(publish_result.is_published())
            # if (not publish_result.is_published()):
            if (publish_result.rc != mqtt.MQTT_ERR_SUCCESS):
                if SCREEN_DISPLAY:
                    print(mqtt.error_string(publish_result.rc))
                json_filename = "tsqueue_testbench_basin.json"
                listObj = []
                # Read json file
                if (os.path.isfile(json_filename)):
                    with open(json_filename) as json_file:
                        listObj = json.load(json_file)
                # Verify existing list
                # print(listObj)
                # print(type(listObj))
                # print(json.dumps(listObj))
                # print(json.dumps(listObj[0]))
                listObj.append(json_data)
                with open(json_filename, 'w') as json_file:
                    json.dump(listObj, json_file, indent=4, separators=(",",": "))
        except (ValueError, RuntimeError) as error:
            if SCREEN_DISPLAY:
                print(error)
        finally:
            if SCREEN_DISPLAY:
                print(publish_result)
                print(json_data)

    if SCREEN_DISPLAY:
        print("")

    if SAVE_TO_FILE:
        fid.write("\r\n")
        if (fid.closed == False):
            fid.close()

    # print("begin to sleep")
    # time.sleep(SLEEP_TIME_SECONDS) # sleep to the next loop
       

except KeyboardInterrupt: # 当try中的代码出现错误或连接数据库等连接问题时，不运行，空跑一下
    pass

finally: # try中的代码不断从设备获取数据，一直到最后终止连接时，才运行finally中的代码
    if (SAVE_TO_FILE and (fid.closed == False)): # 关闭连接
        fid.close()
    if (ard.isOpen()):
        ard.close()
    if (PUBLISH_TO_THINGSBOARD): # 关闭接口
        client.loop_stop()
        client.disconnect()